In [3]:
import pyedflib
import mne
import numpy as np

import torchvision
import torchaudio
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau


import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

from IPython.display import clear_output
from tqdm import tqdm
import matplotlib.ticker as ticker
from os import listdir
import os
import optuna
import logging
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

from dictionary import *

%matplotlib inline

# Collecting Data

In [4]:
def ReadSignal(file_name): 

    f = pyedflib.EdfReader(file_name)
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    sigbufs = np.zeros((20, f.getNSamples()[0])) #or n
    print(file_name, n)
    
    if n == 22:
        for i in np.arange(19):
            sigbufs[i, :] = f.readSignal(i)
        sigbufs[19, :] = f.readSignal(21)
    elif n == 23:
        for i in np.arange(19):
            sigbufs[i, :] = f.readSignal(i)
        sigbufs[19, :] = f.readSignal(20)
    elif n == 21:
        for i in np.arange(20):
            sigbufs[i, :] = f.readSignal(i)
    else:
        for i in np.arange(n):
            sigbufs[i, :] = f.readSignal(i)

    time = [1/f.samplefrequency(0) * i for i in range(len(sigbufs[0]))]

    annotations = f.readAnnotations()  

    new_annot = [(annotations[0][i], annotations[1][i], annotations[2][i])  
                 for i in range(len(annotations[0])) 
                                if (annotations[1][i] > 0) and (annotations[2][i] in ["Ð´Ð°Ð±Ð» Ñ\x81Ð¿Ð°Ð¹Ðº", "*", "?", "F7", 
                                                                                      'Ð\xa03(blockUserBlock)', "F7(blockUserBlock)", 
                                                                                      "O2(blockUserBlock)", "F7(blockUserBlock)", 
                                                                                      'Ð¡4(blockUserBlock)', 'Ñ\x8dÐ°',
                                                                                      "F7(blockUserBlock)(blockUserBlock)"])]
    f.close()
    return sigbufs, new_annot, time, f.samplefrequency(0)

In [5]:
record_names = ALL_RECORD_NAMES

records = []
annots = []
times = []
freqs = []
for file_name in record_names:

    sigbufs, new_annot, time, freq = ReadSignal("data/"+file_name)
    records.append(sigbufs)
    annots.append(new_annot)
    times.append(time)
    freqs.append(freq)
    

data/NNSpecialistsData/1.edf 20
data/NNSpecialistsData/-2 marked.edf 23
data/NNSpecialistsData/patient4.edf 21
data/NNSpecialistsData/5_marked.edf 23
data/MoscowSpecialistsData/P1_3.edf 20
data/MoscowSpecialistsData/P1_4.edf 20
data/MoscowSpecialistsData/P2.edf 20
data/MoscowSpecialistsData/P3.edf 20
data/MoscowSpecialistsData/P4.edf 20
data/MoscowSpecialistsData/P5.edf 20
data/MoscowSpecialistsData/P6.edf 20
data/MoscowSpecialistsData/P7.edf 20
data/MoscowSpecialistsData/P9.edf 20
data/MoscowSpecialistsData/P10.edf 20
data/MoscowSpecialistsData/P11.edf 20
data/MoscowSpecialistsData/P12.edf 20
data/MoscowSpecialistsData/P13.edf 20
data/MoscowSpecialistsData/P14.edf 20
data/MoscowSpecialistsData/P15.edf 20
data/MoscowSpecialistsData/P16.edf 20
data/MoscowSpecialistsData/P17.edf 20
data/MoscowSpecialistsData/P18.edf 20
data/MoscowSpecialistsData/P19.edf 20
data/MoscowSpecialistsData/P23.edf 20
data/MoscowSpecialistsData/P26.edf 20
data/MoscowSpecialistsData/P27.edf 20
data/MoscowSpeciali

In [6]:
def NormalizeAndClip(data):
    for i in tqdm(range(len(data))):
        signal = data[i]
        means = signal.mean(axis=1)[..., None]
        stds = signal.std(axis=1)[..., None]
        signal = np.clip((signal - means) / stds, a_min=-10, a_max=10)
        data[i] = signal

In [7]:
NormalizeAndClip(records)

  0%|                                                                                                                                  | 0/32 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:28<00:00,  1.12it/s]


In [8]:
freqs

[500.0,
 500.0,
 500.0,
 500.0,
 199.50274692224406,
 500.0,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406,
 199.50274692224406]

In [9]:
def Transform(sample_rate): # only for 199
    transform = 0
    new_freq = 0
    if sample_rate == 199:
        transform500 = torchaudio.transforms.Resample(250625, 100000)
        transform1000 = torchaudio.transforms.Resample(50125, 10000)
        
        new_freq = 80000 / 401
    else:
        transform = torchaudio.transforms.Resample(100000, 250625) 
        new_freq = 500
    for i in range(len(records)):
        if int(freqs[i]) != sample_rate:
            if int(freqs[i]) == 500:
                new_sigbufs = []
                sigbufs = records[i]
                for sig in tqdm(sigbufs):
                    new_sigbufs.append(transform500(torch.FloatTensor(sig)))
                new_sigbufs = np.array(new_sigbufs)
                records[i] = new_sigbufs
                freqs[i] = new_freq
                times[i] = [1/new_freq * j for j in range(len(new_sigbufs[0]))]   
            elif int(freqs[i]) == 1000:
                new_sigbufs = []
                sigbufs = records[i]
                for sig in tqdm(sigbufs):
                    new_sigbufs.append(transform1000(torch.FloatTensor(sig)))
                new_sigbufs = np.array(new_sigbufs)
                records[i] = new_sigbufs
                freqs[i] = new_freq
                times[i] = [1/new_freq * j for j in range(len(new_sigbufs[0]))]

In [10]:
Transform(199)

  0%|                                                                                                                                  | 0/20 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 29.58it/s]


In [11]:
for i in range(len(freqs)):
    freqs[i] = 80000 / 401
    times[i] = [1/freqs[i] * j for j in range(len(records[i][0]))]

In [12]:
def Labeling(time, events_lst):
    labels = np.zeros_like(time)

    for events in events_lst:
        for event in tqdm(events):
            start = np.array(time < event[0]).argmin()
            fin = np.array(time < event[0] + event[1]).argmin()
            labels[start:fin] =  1
    return labels
        

In [13]:
all_labels = []

for i in range(len(records)):
    name = "data/" + record_names[i].split("/")[0] + "/Labels/" + record_names[i].split("/")[1] + "freq" + str(freqs[i])[:3]
    if "NN" in record_names[i]:
        all_labels.append(np.load(name + ".npy"))
    elif "Moscow" in record_names[i]:
        all_labels.append(np.load(name + ".npy"))

# Train Test Split

In [14]:
def Shuffle(records):
    shuffled_records = []
    for i in range(len(records)):
        shuffled_records.append(records[i].copy())
        np.random.shuffle(shuffled_records[-1][:-1])
    return shuffled_records

In [15]:
def GetTrainTestByIdxs(array, train_indices):
    train_array = [array[idx] for idx, is_train in enumerate(train_indices) if is_train == True]
    test_array = [array[idx] for idx, is_train in enumerate(train_indices) if is_train == False]    
    return train_array, test_array

def GetTrainTestSplit(records, annots, times, labels, index, shuffle_leads, N, sneos=None, mcs=None):
    train_indices = np.ones(N) #number of records
    if N %2 == 1 and index == N // 2 + 1:
        train_indices[-1] = 0
    else:
        train_indices[2*index: 2*index+2] = 0
        
    train_sneos, test_sneos = None, None
    train_mcs, test_mcs = None, None
    
    train_records, test_records = GetTrainTestByIdxs(records, train_indices)
    if sneos is not None:
        train_sneos, test_sneos = GetTrainTestByIdxs(sneos, train_indices)
        train_mcs, test_mcs = GetTrainTestByIdxs(mcs, train_indices)
        
        
    train_labels, test_labels = GetTrainTestByIdxs(labels, train_indices)  
    train_annots, test_annots = GetTrainTestByIdxs(annots, train_indices)
    train_times, test_times = GetTrainTestByIdxs(times, train_indices)
    
    
    if shuffle_leads:
        train_records = Shuffle(train_records)
    
    train_data = []
    for i in range(len(train_records)):
        train_time_start = train_annots[i][0][0]
        train_time_end = train_annots[i][-1][0]
        train_idx_start = (np.array(train_times[i]) < train_time_start).argmin()
        train_idx_fin = (np.array(train_times[i])< train_time_end).argmin()
        
        if sneos is not None:
            train_data.append((torch.FloatTensor(train_records[i][:, train_idx_start:train_idx_fin]), 
                              torch.FloatTensor(train_sneos[i][:, train_idx_start:train_idx_fin]), 
                              torch.FloatTensor(train_mcs[i][:, train_idx_start:train_idx_fin])))
        else:
            train_data.append((torch.FloatTensor(train_records[i][:, train_idx_start:train_idx_fin]), None, None))
        
        current_labels = train_labels[i][train_idx_start:train_idx_fin]
        new_trainl = torch.zeros(2, len(current_labels))
        new_trainl = (torch.arange(2) == torch.LongTensor(current_labels)[:,None]).T
        new_trainl = new_trainl.float()
        train_labels[i] = new_trainl
    
    test_data = []
    for i in range(len(test_records)):
        test_time_start = test_annots[i][0][0]
        test_time_end = test_annots[i][-1][0]
        test_idx_start = (np.array(test_times[i]) < test_time_start).argmin()
        test_idx_fin = (np.array(test_times[i])< test_time_end).argmin()

        if sneos is not None:
            test_data.append((torch.FloatTensor(test_records[i][:, test_idx_start:test_idx_fin]), 
                              torch.FloatTensor(test_sneos[i][:, test_idx_start:test_idx_fin]), 
                              torch.FloatTensor(test_mcs[i][:, test_idx_start:test_idx_fin])))
        else:
            test_data.append((torch.FloatTensor(test_records[i][:, test_idx_start:test_idx_fin]), None, None))                  
        
        current_labels = test_labels[i][test_idx_start:test_idx_fin]
        new_testl = torch.zeros(2, len(current_labels))
        new_testl = (torch.arange(2) == torch.LongTensor(current_labels)[:,None]).T
        new_testl = new_testl.float()
        test_labels[i] = new_testl
    
    return train_data, train_labels, test_data, test_labels

In [16]:
RECEPTIVE_FIELD = 4000
OVERLAP = 0 

def CreateSamples(x, labels, rf = RECEPTIVE_FIELD, ov = OVERLAP, sneo=None, mc=None):
    inout_seq = []
    L = x.shape[-1]
    if sneo != None:
        for i in tqdm(range(ov, L- rf - ov, rf)):
            train_seq = x[:, i-ov:i+rf+ov]
            train_sneo = sneo[:, i-ov:i+rf+ov]
            train_mc = mc[:, i-ov:i+rf+ov]
            
            train_label = labels[:, i:i+rf]
            inout_seq.append((train_seq, train_sneo, train_mc, train_label))
    else:
        for i in tqdm(range(ov, L- rf - ov, rf)):
            train_seq = x[:, i-ov:i+rf+ov]
            train_label = labels[:, i:i+rf]
            inout_seq.append((train_seq, "None", "None", train_label)) #with real None problems with dataloader
  
    return inout_seq
    

#  Net and Training

In [17]:
class conbr_block(nn.Module):
    def __init__(self, in_layer, out_layer, kernel_size, stride, dilation):
        super(conbr_block, self).__init__()
        self.stride = stride
        self.conv1 = nn.Conv1d(in_layer, out_layer, kernel_size=kernel_size, stride=stride, dilation = dilation, 
                               padding = int(np.ceil(dilation * (kernel_size-1) / 2)), bias=True) # for stride=1, else need to calculate and change
        self.bn = nn.BatchNorm1d(out_layer)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        inp_shape = int(np.ceil(x.shape[2] / self.stride))
        x = self.conv1(x)
        x = self.bn(x)
        out = self.relu(x)[:, :, :inp_shape] 
        #print("conbr_out", out.shape)
        return out      

class se_block(nn.Module):
    def __init__(self,in_layer, out_layer):
        super(se_block, self).__init__()
        
        self.conv1 = nn.Conv1d(in_layer, out_layer//8, kernel_size=1, padding=0)
        self.conv2 = nn.Conv1d(out_layer//8, in_layer, kernel_size=1, padding=0)
        self.fc = nn.Linear(1,out_layer//8)
        self.fc2 = nn.Linear(out_layer//8,out_layer)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self,x):

        x_se = nn.functional.adaptive_avg_pool1d(x,1)
        x_se = self.conv1(x_se)
        x_se = self.relu(x_se)
        x_se = self.conv2(x_se)
        x_se = self.sigmoid(x_se)
        
        x_out = torch.add(x, x_se)
        return x_out

class re_block(nn.Module):
    def __init__(self, in_layer, out_layer, kernel_size, dilation):
        super(re_block, self).__init__()
        
        self.cbr1 = conbr_block(in_layer,out_layer, kernel_size, 1, dilation)
        self.cbr2 = conbr_block(out_layer,out_layer, kernel_size, 1, dilation)
        self.seblock = se_block(out_layer, out_layer)
    
    def forward(self,x):
        x_re = self.cbr1(x)
        x_re = self.cbr2(x_re)
        x_re = self.seblock(x_re)        
        x_out = torch.add(x, x_re)
        return x_out          

class UNET_1D(nn.Module):
    def __init__(self ,input_dim,layer_n,kernel_size, n_down_layers, depth, n_features=1): # n_features for additional features in some other exps
        super(UNET_1D, self).__init__()
        self.input_dim = input_dim
        self.layer_n = layer_n
        self.kernel_size = kernel_size
        self.n_down_layers = n_down_layers
        self.depth = depth
        
        self.AvgPool1D = nn.ModuleList([nn.AvgPool1d(input_dim, stride=5**i, padding=8) for i in range(1, self.n_down_layers)])
        
        
        self.layer1 = self.down_layer(self.input_dim, self.layer_n, self.kernel_size,1, depth)
        self.layer1_sneo = self.down_layer(self.input_dim, self.layer_n, self.kernel_size,1, self.depth)
        self.layer1_mc = self.down_layer(self.input_dim, self.layer_n, self.kernel_size,1, self.depth)
        
        self.layer2 = self.down_layer(self.layer_n, int(self.layer_n*2), self.kernel_size,5, self.depth)
        
        self.down_layers = nn.ModuleList([self.down_layer(int(self.layer_n*(1+i))+n_features*int(self.input_dim), int(self.layer_n*(2+i)), 
                                            self.kernel_size,5, self.depth) for i in range(1, self.n_down_layers)])


        self.cbr_up = nn.ModuleList([conbr_block(int(self.layer_n*(2*i+1)), int(self.layer_n*i), self.kernel_size, 1, 1) 
                       for i in range(self.n_down_layers, 0, -1)]) #input size is a sizes sum of outs of 2 down layers for current down depth
        self.upsample = nn.Upsample(scale_factor=5, mode='nearest') 
        
        self.outcov = nn.Conv1d(self.layer_n, 2, kernel_size=self.kernel_size, stride=1,
                                padding = int(np.ceil(1 * (self.kernel_size-1) / 2)))
    
        
    def down_layer(self, input_layer, out_layer, kernel, stride, depth):
        block = []
        block.append(conbr_block(input_layer, out_layer, kernel, stride, 1))
        for i in range(depth):
            block.append(re_block(out_layer,out_layer,kernel,1))
        return nn.Sequential(*block)
        
        
            
    def forward(self, x):
        inp_shape = x.shape[2]
        
        
        
        #############Encoder#####################

        out_0 = self.layer1(x)
        out_1 = self.layer2(out_0)
        outs = [out_0, out_1]
        for i in range(self.n_down_layers-1):
            pool = self.AvgPool1D[i](x)
            x_down = torch.cat([outs[-1],pool],1)

            outs.append(self.down_layers[i](x_down))




        #############Decoder####################
        up = self.upsample(outs[-1])[:, :, :outs[-2].shape[2]]
        for i in range(self.n_down_layers):
                        
            up = torch.cat([up,outs[-2-i]],1)
            up = self.cbr_up[i](up)
            if i + 1 < self.n_down_layers:
                up = self.upsample(up)[:, :, :outs[-3-i].shape[2]]

        out = self.outcov(up)


        return out[:, :, :inp_shape] 

In [19]:
def run_epoch(model, optimizer, criterion, dataloader, is_training=False):
    epoch_loss = 0

    if is_training:
        model.train()
    else:
        model.eval()

    for idx, (x, sneo, mc, y) in enumerate(dataloader):
        if is_training:
            optimizer.zero_grad()
        out = None
        if sneo[0] != "None":
            
            out = model(x.to('cuda'), sneo.to("cuda"), mc.to("cuda"))
        else:
            #x, y = sample
            out = model(x.to("cuda"))

        loss = criterion(out, y.to('cuda'))

        if is_training:
            loss.backward()
            optimizer.step()

        epoch_loss += (loss.detach().item() / len(dataloader))


    return epoch_loss

In [20]:
def GetRawMetricsAndCMEPINoLogging(predictions, test_labels):
    TP_sum = 0
    FP_sum = 0
    FN_sum = 0    
    
    for i in range(len(test_labels)):
        pred_len = len(predictions[i])

        TP, FP, FN = CollectingTPFPFN(predictions[i], test_labels[i][1, :pred_len].numpy())

        TP_sum += TP
        FP_sum += FP
        FN_sum += FN      

        
    return 2 * TP_sum / (2 * TP_sum + FP_sum + FN_sum)
    

In [21]:
def CalculateMetric(model, test_dataloader, threshold = 20):
    

    all_preds = []
    record_preds = []
    answers = []
    all_answers = []
    
    for idx, (x, sneo, mc, y) in enumerate(test_dataloader):
        #print("X shape", x.shape)
        out = None
        if sneo[0] != "None":
            out = model(x.to('cuda'), sneo.to("cuda"), mc.to("cuda"))
        else:
            out = model(x.to("cuda"))
        
        m = nn.Softmax(dim=1)
        out = m(out)
            
        preds = np.argmax(out.detach().cpu().numpy(), axis=1)
        record_preds.append(preds)
        answers.append(y.detach().cpu().numpy()[:, 1])
    shapes = np.array(record_preds).shape
    #print("Shapes", shapes)
    record_preds = np.array(record_preds).reshape(shapes[0] * shapes[1] * shapes[2])
    answers = torch.LongTensor(np.vstack([np.zeros(shapes[0] * shapes[1] * shapes[2]), 
                         np.array(answers).reshape(shapes[0] * shapes[1] * shapes[2])]))
    
    all_preds.append(record_preds)
    all_answers.append(answers)
    
    
    threshold1 = threshold
    threshold2 = None

    
    for j in range(len(all_preds)):
        PostProcessing(all_preds[j], threshold1, threshold2)
    metric = GetRawMetricsAndCMEPINoLogging(all_preds, all_answers)
    return metric

In [22]:
def Train(model,
          train_dataloader,
          test_dataloader,
          i,
          path, threshold = 20,
          NeedsPathing=True):

    criterion = nn.BCEWithLogitsLoss(
    )  #pos_weight = torch.FloatTensor([[0.3, 0.7]] * 4000).T.to("cuda"))
    optimizer = optim.Adam(model.parameters(),
                           lr=0.001,
                           betas=(0.9, 0.999),
                           eps=1e-9)
    scheduler = ReduceLROnPlateau(optimizer,
                                  patience=3,
                                  factor=0.5,
                                  min_lr=0.00001)
    epochs = 50

    losses_train = []
    losses_test = []
    metrics = []
    best_loss = 10e9
    best_metric = 0

    # begin training

    early_stop_count = 15
    current_es = 0
    best_epoch = 0
    for epoch in range(epochs):

        loss_train = run_epoch(model,
                               optimizer,
                               criterion,
                               train_dataloader,
                               is_training=True)
        loss_val = run_epoch(model, optimizer, criterion, test_dataloader)
        scheduler.step(loss_val)
        losses_train.append(loss_train)
        losses_test.append(loss_val)

        metric = CalculateMetric(model, test_dataloader, threshold)

        metrics.append(metric)
        


        if metric >= best_metric:
            best_metric = metric
            torch.save(model.state_dict(),
                    path + "/Split" + str(i) + "/Unet1d")
            best_epoch = epoch
        
        if (NeedsPathing):
            clear_output(True)
            fig = plt.figure(figsize=(10, 9))

            ax_1 = fig.add_subplot(3, 1, 1)
            ax_2 = fig.add_subplot(3, 1, 2)
            ax_3 = fig.add_subplot(3, 1, 3)

            ax_1.set_title('train loss')
            ax_1.plot(losses_train)
            ax_2.set_title('test loss')
            ax_2.plot(losses_test)
            ax_3.set_title('test metric')
            ax_3.plot(metrics)
            plt.savefig(path + "/Split" + str(i) + "/Unet1dFigure")

            plt.show()

        f = open(path + "/Split" + str(i) + "/BestEpoch.txt", 'w')
        f.write(str(best_epoch))
        f.close()
        if (epoch % 10 == 0):
            print('Epoch[{}/{}] | loss train:{:.6f}, test:{:.6f}'.format(
                epoch + 1, epochs, loss_train, loss_val))

    return best_metric


# Collecting Predictions, Calculating Metrics

In [23]:
def CollectingPreds(model, test_data):

    model.eval()
    model.cpu()
    all_preds = []
    for i in range(len(test_data)):
        record_preds = []
        for idx in tqdm(range(OVERLAP, test_data[i][0].size()[1]- RECEPTIVE_FIELD - OVERLAP, RECEPTIVE_FIELD)):

            test_seq = test_data[i][0][:, idx-OVERLAP:idx+RECEPTIVE_FIELD+OVERLAP][None, ...]
            out = None
            if test_data[i][1] is not None:
                test_sneo_seq = test_data[i][1][:, idx-OVERLAP:idx+RECEPTIVE_FIELD+OVERLAP][None, ...]
                test_mc_seq = test_data[i][2][:, idx-OVERLAP:idx+RECEPTIVE_FIELD+OVERLAP][None, ...] 
                out = model(test_seq, test_sneo_seq, test_mc_seq)
            else:
                out = model(test_seq)
                   
            m = nn.Softmax(dim=1)
            out = m(out)
            
            preds = np.argmax(out.detach().cpu().numpy(), axis=1)
            record_preds.append(preds)
        shapes = np.array(record_preds).shape
        record_preds = np.array(record_preds).reshape(shapes[0] * shapes[1] * shapes[2])
        all_preds.append(record_preds)
    return all_preds

In [24]:
def MergeClose(predictions, threshold):
    i = 0
    in_event = False
    while i < len(predictions):
        while i < len(predictions) and predictions[i] == 1:
            in_event = True
            i += 1
        if  i < len(predictions) and in_event:
            if np.any(predictions[i:i+threshold]):
                while  i < len(predictions) and predictions[i] == 0:
                    predictions[i] = 1
                    i += 1
            else:
                in_event = False
        i += 1

def DeleteShortEvents(predictions, threshold):
    i = 0
    while i < len(predictions):
        event_len = 0
        event_idx_start = i
        while i < len(predictions) and predictions[i] == 1:
            i += 1
            event_len += 1
        if event_len < threshold:
            predictions[event_idx_start:i] = 0
        i += 1
def PostProcessing(predictions, threshold1, threshold2=None):
    MergeClose(predictions, threshold1)
    if threshold2 is None:
        DeleteShortEvents(predictions, threshold1)
    else:
        DeleteShortEvents(predictions, threshold2)

In [25]:
def CollectingTPFPFN(pred_labels, true_labels):
    i = 0
    TP = 0
    FP = 0
    FN = 0

    is_true_flag = 0
    is_pred_flag = 0
    is_used_pred_flag = 0
    
    while i < len(pred_labels):
        if pred_labels[i] == 0:
            is_used_pred_flag = 0
        while i < len(pred_labels) and true_labels[i] == 1:
            is_true_flag = 1
            if not is_used_pred_flag:
                if pred_labels[i] == 1:
                    is_pred_flag = 1
                    is_used_pred_flag = 1 
            else:
                if pred_labels[i] == 0:
                    is_used_pred_flag = 0
            i += 1
        if is_true_flag:
            if is_pred_flag:
                TP += 1
            else:
                FN += 1
            i -= 1

        
        is_true_flag = 0
        is_pred_flag = 0   
        i += 1

    i = 0
    while i < len(pred_labels):
        while i < len(pred_labels) and pred_labels[i] == 1:
            is_pred_flag = 1
            if true_labels[i] == 1:
                is_true_flag = 1
            i += 1
        if is_pred_flag and not is_true_flag:
            FP += 1
        is_pred_flag = 0
        is_true_flag = 0
        i += 1

    return TP, FP, FN 


def GetRawMetricsAndCMEPI(predictions, test_labels, split_index, path, N):
    lens = []
    sums = []

    acc = []
    precision = []
    recall = []
    f1 = []

    TP_sum = 0
    FP_sum = 0
    FN_sum = 0
    
    all_cm = 0
    
    train_indices = np.ones(N) #number of records
    if N %2 == 1 and split_index == N // 2 + 1:
        train_indices[-1] = 0
    else:
        train_indices[2*split_index: 2*split_index+2] = 0
        
    train_record_names, test_record_names = GetTrainTestByIdxs(record_names, train_indices)
    
        
    f = open(path + "/Split" + str(split_index) + "/Metrics.txt", 'w')
    
    for i in range(len(test_labels)):
        pred_len = len(predictions[i])

        TP, FP, FN = CollectingTPFPFN(predictions[i], test_labels[i][1, :pred_len].numpy())

        TP_sum += TP
        FP_sum += FP
        FN_sum += FN
            

        if TP + FP != 0:
            precision.append(TP / (TP + FP))
        else:
            precision.append(0)
        recall.append(TP / (TP + FN))
        f1.append(2 * TP / (2 * TP + FP + FN))
        

        cm = np.array([[0, FP], [FN, TP]])
        all_cm += cm
        
        f.write("=============Record " + test_record_names[i] + "================\n")
    
        f.write("precision " + str(precision[i]) + "\n")
        f.write("recall " + str(recall[i]) + "\n")
        f.write("f1 score " + str(f1[i]) + "\n")
                
        con_mat = ConfusionMatrixDisplay(cm)
        con_mat.plot().figure_.savefig(path + "/Split" + str(split_index) + 
                                "/ConfusionMatrix_" + test_record_names[i].replace("/", "_") + ".png")
    f.write("===========ALL RECORDS SCORE==================\n")
    
    
    if TP_sum + FP_sum != 0:
        f.write("Full precision " + str(TP_sum / (TP_sum + FP_sum)) + "\n")
    else:
        f.write("Full precision " + str(0) + "\n")
    f.write("Full recall " + str(TP_sum / (TP_sum + FN_sum)) + "\n")
    f.write("Full f1 " + str(2 * TP_sum / (2 * TP_sum + FP_sum + FN_sum)) + "\n")
    
    
    f.close()
    con_mat = ConfusionMatrixDisplay(all_cm)
    con_mat.plot().figure_.savefig(path + "/Split" + str(split_index) + 
                                "/ConfusionMatrixFull.png")

In [26]:
def CalculateSWI(labels, sampling_rate=500, area_SWI=False):
    if not area_SWI:
        event_num = 0
        silence_num = 0
        for i in range(0, len(labels), sampling_rate):
            if labels[i:i+sampling_rate].max() == 1:
                event_num += 1
            else:
                silence_num += 1
        return event_num / (event_num + silence_num)
    return labels.sum() / len(labels)

# Logging and Creating Folder

In [27]:
def  LogResults(model, test_data, test_labels, i, path, last_epoch, low_freq, write_edf, annots, area_SWI, N, threshold = 30):
    if not last_epoch and test_data[0][1] is not None:
        model.load_state_dict(torch.load(path + "/Split" + str(i) +"/Unet1d"))  
    elif not last_epoch:
        model.load_state_dict(torch.load(path + "/Split" + str(i) +"/Unet1d"), strict=False)
    all_preds = CollectingPreds(model, test_data)
    sr = 200
    if low_freq: 
        threshold= 20
    if not low_freq:
        sr = 500
    for j in range(len(all_preds)):
        PostProcessing(all_preds[j], threshold)
    GetRawMetricsAndCMEPI(all_preds, test_labels, i, path, N)  
    
    if write_edf:
        WriteEDF(all_preds, i, annots, path, N)
        
    SWIs_pred = []
    SWIs_true = []
    
    for j in range(len(all_preds)):
        SWIs_pred.append(CalculateSWI(all_preds[j], sr, area_SWI))
        SWIs_true.append(CalculateSWI(test_labels[j][1], sr, area_SWI))     
    np.savetxt(path + "/Split" + str(i) +"/SWIPred", np.array(SWIs_pred))
    np.savetxt(path + "/Split" + str(i) +"/SWITrue", np.array(SWIs_true))    

In [28]:
def CreateFolder(path, n_splits):
    try:  
        os.mkdir(path)  
    except OSError as error:
        True

    for i in range(n_splits):
        try:
            os.mkdir(path + "/Split" + str(i))
        except OSError as error:
            continue
    

# Write EDF

In [29]:
def CreateNewAnnotation(time_start, labels, freq): 
    freq = 1/freq
    i = 0
    label_starts = [time_start]
    label_lens = [-1]
    desc = ["StartPredictionTime"]
    while i < len(labels):
        if labels[i] == 1:
            desc.append("ModelPrediction")
            label_starts.append(time_start + i*freq)
            cur_start = i
            while i < len(labels) and labels[i] == 1:
                i += 1
            label_lens.append((i - cur_start) * freq)
        i += 1
    label_starts += [time_start + i*freq]
    label_lens += [-1]
    desc += ["EndPredictionTime"]

    return np.array(label_starts), np.array(label_lens), np.array(desc)

In [30]:
def WriteEDF(predictions, split_index, annots, path, N):
    # Функция записывает в цикле все записии в edf с предсказаниями сети
    freq = 80000 / 401
    train_indices = np.ones(N) #number of records
    
    if N %2 == 1 and split_index == N // 2 + 1:
        train_indices[-1] = 0
    else:
        train_indices[2*split_index: 2*split_index+2] = 0
        
    train_record_names, test_record_names = GetTrainTestByIdxs(record_names, train_indices)

    train_annots, test_annots = GetTrainTestByIdxs(annots, train_indices)
    for i in range(len(test_record_names)):
        time_start = test_annots[i][0][0]
        preds_annotations = CreateNewAnnotation(time_start, predictions[i], freq)
        # считывание
        data = mne.io.read_raw_edf("data/" + test_record_names[i])

        # Обработка аннотации для записи в файл
        preds_annotations = list(preds_annotations)
        preds_annotations[1] = np.clip(preds_annotations[1], a_min=0, a_max = None)

        old_annot = np.array([[data.annotations[i]["onset"], data.annotations[i]["duration"], data.annotations[i]["description"]] 
                      for i in range(len(data.annotations))])
        
        full_annot = np.concatenate([np.array(preds_annotations), old_annot.T], axis=1)
        annotations = mne.Annotations(np.array(full_annot)[0], np.array(full_annot)[1], np.array(full_annot)[2])
        data.set_annotations(annotations)
        
        # Экспорт
        data.export(path + "/Split" + str(split_index) + "/Preds_" + test_record_names[i].split("/")[1], overwrite=True)
        data.close()
        

# Experements

In [31]:
def model_train(threshold,
                window,
                records,
                labels,
                annots,
                times,
                layer_n,
                kernel_size,
                n_down_layers,
                depth,
                shuffle_leads=False,
                sneos=None,
                mcs=None,
                is_train=True):
    path = "OptunaTraining"
    N = len(records)
    n_splits = N // 2
    if N % 2 == 1:
        n_splits += 1
    CreateFolder(path, n_splits)
    result_metric = []
    for i in range(n_splits):
        train_data, train_labels, test_data, test_labels = GetTrainTestSplit(
            records, annots, times, labels, i, shuffle_leads, N, sneos, mcs)
        train_samples = []
        for j in range(len(train_data)):
            train_samples += CreateSamples(train_data[j][0],
                                           train_labels[j],
                                           sneo=train_data[j][1],
                                           mc=train_data[j][2],
                                           rf=window)
            #train_samples += CreateSamples(train_data[j][0], train_labels[j], sneo=train_data[j][1], mc=train_data[j][2])

        test_samples = []
        for j in range(len(test_data)):
            test_samples += CreateSamples(test_data[j][0],
                                          test_labels[j],
                                          sneo=test_data[j][1],
                                          mc=test_data[j][2],
                                          rf=window)
            #test_samples += CreateSamples(test_data[j][0], test_labels[j], sneo=test_data[j][1], mc=test_data[j][2])

        train_dataloader = DataLoader(
            train_samples, batch_size=16, shuffle=True, drop_last=True
        )  # or train_samples for 4000 or new_train_samples for 100
        test_dataloader = DataLoader(test_samples,
                                     batch_size=4,
                                     shuffle=False,
                                     drop_last=True)

        TrainModel = UNET_1D(20, layer_n, kernel_size, n_down_layers, depth, 1)
        TrainModel = TrainModel.to("cuda")
        if is_train:
            result_metric.append(Train(TrainModel,
                                train_dataloader,
                                test_dataloader,
                                i,
                                path,threshold, NeedsPathing=False))
    return np.mean(result_metric)
        #LogResults(TrainModel, test_data, test_labels, i, path, False, False, False, annots, False, N, threshold)

In [ ]:
def objective(trial):
    torch.cuda.empty_cache()
    window = trial.suggest_int('window', 2000, 8000, step=5)
    layer_n = trial.suggest_int('layer_n', 96, 168)
    kernel_size = trial.suggest_int('kernel_size', 3, 15)
    n_down_layers = trial.suggest_int('n_down_layers', 2, 6)
    depth = trial.suggest_int('depth', 2, 6)
    threshold =trial.suggest_int('threshold', 20, 50)
    TrainMetric = model_train(threshold, window, records, all_labels, annots,
                                          times, layer_n, kernel_size,
                                          n_down_layers, depth)
    return TrainMetric

logger = logging.getLogger()

logger.setLevel(logging.INFO)
logger.addHandler(logging.FileHandler("OptunaTraining/OptunaLog.txt", mode="w"))

optuna.logging.enable_propagation()
optuna.logging.disable_default_handler()

study = optuna.create_study(direction="maximize", sampler=optuna.samplers.RandomSampler())

logger.info("Start optimization.")

study.optimize(objective, n_trials=200)

with open("OptunaTraining/foo.log") as f:
    assert f.readline().startswith("A new study created")
    assert f.readline() == "Start optimization.\n"

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 47127.01it/s]


Epoch[1/50] | loss train:0.423614, test:0.425572
Epoch[11/50] | loss train:0.144966, test:0.233183
Epoch[21/50] | loss train:0.078964, test:0.217114
Epoch[31/50] | loss train:0.055344, test:0.286891
Epoch[41/50] | loss train:0.047652, test:0.297482


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 43345.39it/s]


Epoch[1/50] | loss train:0.417377, test:0.509344
Epoch[11/50] | loss train:0.128793, test:0.637698
Epoch[21/50] | loss train:0.090549, test:0.688735
Epoch[31/50] | loss train:0.078645, test:0.765088
Epoch[41/50] | loss train:0.072808, test:0.762257


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 25620.97it/s]


...

Epoch[1/50] | loss train:0.436242, test:0.428158
Epoch[11/50] | loss train:0.134048, test:0.485750
Epoch[21/50] | loss train:0.088200, test:0.494681
Epoch[31/50] | loss train:0.075474, test:0.549716
Epoch[41/50] | loss train:0.072663, test:0.560409


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 57763.94it/s]


Epoch[1/50] | loss train:0.430009, test:0.470652
Epoch[11/50] | loss train:0.139007, test:0.143017
Epoch[21/50] | loss train:0.085257, test:0.201568
Epoch[31/50] | loss train:0.066796, test:0.224003
Epoch[41/50] | loss train:0.060380, test:0.221055


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 72967.92it/s]


Epoch[1/50] | loss train:0.448398, test:0.581283
Epoch[11/50] | loss train:0.146463, test:0.124713
Epoch[21/50] | loss train:0.095098, test:0.145646
Epoch[31/50] | loss train:0.066258, test:0.144212
